In [4]:
import sys
print(sys.version)

In [2]:
sg.theme_list()

['Black',
 'BlueMono',
 'BluePurple',
 'BrightColors',
 'BrownBlue',
 'Dark',
 'Dark2',
 'DarkAmber',
 'DarkBlack',
 'DarkBlack1',
 'DarkBlue',
 'DarkBlue1',
 'DarkBlue10',
 'DarkBlue11',
 'DarkBlue12',
 'DarkBlue13',
 'DarkBlue14',
 'DarkBlue15',
 'DarkBlue16',
 'DarkBlue17',
 'DarkBlue2',
 'DarkBlue3',
 'DarkBlue4',
 'DarkBlue5',
 'DarkBlue6',
 'DarkBlue7',
 'DarkBlue8',
 'DarkBlue9',
 'DarkBrown',
 'DarkBrown1',
 'DarkBrown2',
 'DarkBrown3',
 'DarkBrown4',
 'DarkBrown5',
 'DarkBrown6',
 'DarkBrown7',
 'DarkGreen',
 'DarkGreen1',
 'DarkGreen2',
 'DarkGreen3',
 'DarkGreen4',
 'DarkGreen5',
 'DarkGreen6',
 'DarkGreen7',
 'DarkGrey',
 'DarkGrey1',
 'DarkGrey10',
 'DarkGrey11',
 'DarkGrey12',
 'DarkGrey13',
 'DarkGrey14',
 'DarkGrey15',
 'DarkGrey2',
 'DarkGrey3',
 'DarkGrey4',
 'DarkGrey5',
 'DarkGrey6',
 'DarkGrey7',
 'DarkGrey8',
 'DarkGrey9',
 'DarkPurple',
 'DarkPurple1',
 'DarkPurple2',
 'DarkPurple3',
 'DarkPurple4',
 'DarkPurple5',
 'DarkPurple6',
 'DarkPurple7',
 'DarkRed',
 'Da

In [21]:
import PySimpleGUI as sg
import os
import subprocess
import threading 
from pymol import cmd
import shutil

sg.theme("SystemDefault")  # No gray windows please!
# 컴포넌트 정의
# STEP 1 define the layout


left_column = [
    [sg.Image(r"sample_img.png", size=(200, 200))],
    [sg.Text("리셉터 파일을 업로드", size=(30, 1), justification="center")],
    [sg.Input("파일 경로 입력...", size=(25, 1)), sg.FileBrowse(button_text="찾아보기...",key='file_directory_btn')],
    [sg.Button("업로드하기", key = 'file_upload_btn')],
    [sg.Text("업로드된 리셉터 파일 이름 : "), sg.Text("----", key = 'receptor_name')],
    [sg.Text("_" * 35)],
    [sg.Text("추천되는 파라미터", size=(30, 1), justification="center")],
    [
        sg.Text(
            "exhaustiveness : 8 \n Grid 8 8 8 ", size=(30, 3), justification="center"
        )
    ],
    [sg.Table()]
]


right_column = [
    [sg.Text("col Row 1")],
    [
        sg.Frame(
            "My Frame Title",
            [
                [sg.T("리간드 파일 선택")],
                [
                    sg.Column(
                        [
                            [sg.Image(r"folder_icon.png", size=(60, 60))],
                            [sg.Text("리셉터 폴더 선택")],
                        ],
                        element_justification="center",
                    ),
                    sg.Column(
                        [
                            [sg.Image(r"db_icon.png", size=(60, 60))],
                            [sg.Text("온라인 DB 사용")],
                        ],
                        element_justification="center",
                    ),
                ],
                [
                    sg.Input("폴더 경로 입력...", size=(25, 1)),
                    sg.FolderBrowse(button_text="찾아보기..."),
                ],
            ],
        )
    ],
    [
        sg.Table(
            [["Dapagliflozin (1)", "대기중", "--:--:--", "--", "--"], [4, 5, 6, 7, 8]],
            ["리간드 이름", "Docking 수행", "Docking 수행 시간", "결과 평가", "사용한 파라미터"],
            num_rows=2,
            justification="center",
        )
    ],
    [sg.Text("What you print will display below:")],
    [sg.Output(size=(50, 10), key="-OUTPUT-")],
    [sg.In(key="-IN-")],
    [sg.Button("Run Vina"), sg.Button('Eval RMSD'), sg.Button("Clear")],
]


layout = [
    [
        sg.Column(left_column, expand_y=True),
        sg.VerticalSeparator(pad=None),
        sg.Column(right_column, expand_y=True),
    ],
    [sg.Button("debug Button"), sg.Button("Exit")],
]

column = [[layout]]
# STEP 2 - create the window
window = sg.Window("My new window", column, grab_anywhere=True)

def performDocking():
    command = "vina --config config2.txt --log log.txt --out output.pdbqt"
    folder = ".\\SGLT_sample\\"
    p = subprocess.Popen(command, shell=True, stdout=subprocess.PIPE, stderr=subprocess.STDOUT, cwd=folder)
    for line in p.stdout:
        print(line.decode('cp949').strip())
    ## 결과 파일 스플릿 하기 ##
    command = "vina_split --input output.pdbqt"
    p = subprocess.Popen(command, shell=True, stdout=subprocess.PIPE, stderr=subprocess.STDOUT, cwd=folder)
    for line in p.stdout:
        print(line.decode('cp949').strip())

def eval_RMSD():
    cmd.cd('.\SGLT_sample')
    cmd.load('output_ligand_5.pdbqt')
    cmd.load('output_ligand_6.pdbqt')
    rms = cmd.pair_fit('output_ligand_5', 'output_ligand_6')
    print(rms)
    cmd.cd('../')

def upload_file():
    src_file = values['file_directory_btn'] # 
    dst_dir = './uploaded_file/'
    if(src_file != ''): # 파일 선택여부 검사
        fileName = os.path.basename(src_file) # 파일명
        fileType = os.path.splitext(src_file)[1] # 파일타입
        if(fileType == '.pdbqt'):
            shutil.copy(src_file, dst_dir) # 파일 복사
            window['receptor_name'].update(fileName)
            return "파일을 업로드하였습니다. : " + fileName + fileType
        else:
            sg.popup('파일 확장자가 pdbqt가 아닙니다.', title = '오류' )
            return "파일 확장자가 pdbqt가 아닙니다."
    else:
        sg.popup('파일이 선택되지 않았습니다.', title='오류')
        return "파일 경로를 선택해주세요."
# STEP3 - the event loop
while True:
    ( event, values ) = window.read()  # Read the event that happened and the values dictionary
    #print(event, values)
    if event in ( None, "Exit",):  # If user closeddow with X or if user clicked "Exit" button then exit
        break
    if event == "debug Button":
        #f = os.popen("dir")
        #print("You pressed the button" + f.read())
        print(values['file_directory_btn'])
    if event == "file_upload_btn":
        print(upload_file())
    if event == 'Run Vina':
        t = threading.Thread(target=performDocking)
        t.start()
    if event == 'Eval RMSD':
        t = threading.Thread(target=eval_RMSD)
        t.start()
    if event == 'Clear':
        window['-OUTPUT-'].update('')
window.close()


In [2]:
from pymol import cmd

cmd.load('output_ligand_1.pdbqt')
cmd.load('output_ligand_2.pdbqt')

rms = cmd.pair_fit('output_ligand_1', 'output_ligand_2')

print(rms)

2.3762881755828857


In [1]:
print('kerneltest')

kerneltest


In [6]:
import shutil

src_file = 'C:/Users/jundo/Desktop/Git/Graduation_project/db_icon.png'
dst_dir = './db_icon2.png'

shutil.copy(src_file, dst_dir)

'./db_icon2.png'

In [7]:
import PySimpleGUI as sg

layout = [[sg.Text('Upload a file')],
          [sg.Input(key='file'), sg.FileBrowse()],
          [sg.Button('Upload'), sg.Button('Exit')],
          [sg.Listbox(values=[], size=(40, 10), key='listbox')]]

window = sg.Window('Window Title', layout)

while True:
    event, values = window.read()
    if event == sg.WIN_CLOSED or event == 'Exit':
        break
    elif event == 'Upload':
        filename = values['file']
        if filename:
            window['listbox'].update(values=[*values['listbox'], filename])
        else:
            sg.popup('Please choose a file')

window.close()